In [ ]:
# Reference:
#    https://peterroelants.github.io/posts/gaussian-process-kernels/
#    https://distill.pub/2019/visual-exploration-gaussian-processes/
#    http://gregorygundersen.com/blog/2019/06/27/gp-regression/
#
import numpy as np
from numpy.linalg import inv, det, cholesky
from numpy.linalg import solve as backsolve

import jax
from jax import grad, jit, vmap, device_put
import jax.numpy as jnp
import jax.numpy.linalg as jnp_linalg
from jax.experimental import optimizers
import itertools

import matplotlib.pyplot as plt
import matplotlib as mpl
# https://matplotlib.org/3.1.1/gallery/style_sheets/style_sheets_reference.html
mpl.rcParams['lines.linewidth'] = 3
mpl.rcParams['font.size'] = 25
mpl.rcParams['font.family'] = 'Times New Roman'
cmap = plt.cm.get_cmap('bwr')

import sys
sys.path.append('../kernel')
from jaxkern import (rbf_kernel, linear_kernel, cov_se, cov_rq, cov_pe, cdist_sqeuclidean)

from plt_utils import plt_savefig, plt_scaled_colobar_ax
from gp import gp_regression_chol, run_sgd


In [ ]:
# make a dataset with two outputs, correlated, heavy-tail noise. One has more noise than the other.
X1 = np.random.rand(100, 1)  # Observed locations for first output
X2 = np.random.rand(50, 1) * 0.5  # Observed locations for second output

Y1 = np.sin(6 * X1) + np.random.randn(*X1.shape) * 0.03
Y2 = np.sin(6 * X2 + 0.7) + np.random.randn(*X2.shape) * 0.1

plt.figure(figsize=(8, 4))
plt.plot(X1, Y1, "x", mew=2)
_ = plt.plot(X2, Y2, "x", mew=2)

In [ ]:
# Augment the input with ones or zeros to indicate the required output dimension
X_augmented = np.vstack((np.hstack((X1, np.zeros_like(X1))), np.hstack((X2, np.ones_like(X2)))))

# Augment the Y data with ones or zeros that specify a likelihood from the list of likelihoods
Y_augmented = np.vstack((np.hstack((Y1, np.zeros_like(Y1))), np.hstack((Y2, np.ones_like(Y2)))))

X_augmented.shape, Y_augmented.shape

In [ ]:
B = np.array([[2.8,2.65],[2.65,4.45]])

# B[]
# B[]
np.take(B, np.vstack((X_augmented[:,1],Y_augmented[:,1])).astype(np.int).T)




In [ ]:

k = lambda X, Y: cov_se(X[:,0], Y[:,0], ℓ=ℓ) * B[X[:,1],Y[:,1]]


μ, Σ, mll = gp_regression_chol(X_train, y_train, X_test, k, σn)
std = np.expand_dims(np.sqrt(np.diag(Σ)), 1)

ax = axs[i, j]
ax.plot(X_test, μ, color='k')
ax.fill_between(X_test.squeeze(), (μ-2*std).squeeze(), (μ+2*std).squeeze(), alpha=.2, color=cmap(.3))
ax.scatter(X_train, y_train, marker='x', color='r', s=50)
ax.grid()
ax.set_xlim(xlim)
ax.set_ylim(ylim)
ax.set_title('$-\log p(\mathbf{y}\mid X)$'+f'={-mll:.2f}')


In [ ]:

## Parameters 

xlim = (-2, 2)
ylim = (-3, 3)
n_train = 3
n_test = 100
σn = .1
ℓ = 1
train_sizes = [3, 10, 100]
lr = .002
num_steps = 10

def f1(x, σn=.1):
    return np.sin(x)+np.sin(x*5)+np.cos(x*3)

def log_func(f, params):
    print(f"loss={f(params):.3f}\t"
          f"ℓ={params['ℓ']:.3f}\t")
## Plotting 


np.random.seed(0)
X_test = np.expand_dims(np.linspace(*xlim, n_test), 1)
X_train_all = np.expand_dims(
    np.random.uniform(xlim[0], xlim[1], size=np.max(train_sizes)), 1)
ϵ_all = σn*np.random.randn(np.max(train_sizes), 1)


# n_train = 100
X_train = X_train_all[:n_train]
ϵ = ϵ_all[:n_train]
y_train = f_gen(X_train) + ϵ

plt.scatter(X_train, y_train)


In [ ]:

fig, axs = plt.subplots(1, 1)
fig.set_size_inches(5, 5)



for i, ℓ in enumerate(ℓs):
    for j, n_train in enumerate(train_sizes):
        
        X_train = X_train_all[:n_train]
        ϵ = ϵ_all[:n_train]
        y_train = f_gen(X_train) + ϵ
        
        if i == 1:
            jX, jy, jXstar = device_put(X_train), device_put(y_train), device_put(X_test)
            def nmll(params):
                k = lambda X, Y: cov_se(X, Y, ℓ=params['ℓ'])
                μ, Σ, mll = gp_regression_chol(jX, jy, jXstar, k, σn=σn)
                return -mll
            params = {'ℓ': 1.}
            res = run_sgd(nmll, params, lr=lr, num_steps=num_steps, log_func=None)
            ℓ = res['ℓ'].item()
        else:
            ℓ = ℓs[i]
            σn = σns[i]

        k = lambda X, Y: cov_se(X, Y, ℓ=ℓ)
        μ, Σ, mll = gp_regression_chol(X_train, y_train, X_test, k, σn)
        std = np.expand_dims(np.sqrt(np.diag(Σ)), 1)

        ax = axs[i, j]
        ax.plot(X_test, μ, color='k')
        ax.fill_between(X_test.squeeze(), (μ-2*std).squeeze(), (μ+2*std).squeeze(), alpha=.2, color=cmap(.3))
        ax.scatter(X_train, y_train, marker='x', color='r', s=50)
        ax.grid()
        ax.set_xlim(xlim)
        ax.set_ylim(ylim)
        ax.set_title('$-\log p(\mathbf{y}\mid X)$'+f'={-mll:.2f}')
        
        if j == 0 or i == 1:
            ax.set_ylabel(f"ℓ={ℓ:.2f}", fontsize=45)
            
        if i == 2:
            ax.set_xlabel("$n$"+f"={n_train}", fontsize=45)

fig.tight_layout()
# plt_savefig(fig, 'summary/assets/plt_mtgp.png')